In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from abc import ABC, abstractmethod

class BaseRegression(ABC):
    """Abstract Base Class for Regression Models."""

    def __init__(self, learning_rate=0.01, n_iters=500, alpha=0.0, l1_ratio=0.5):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.alpha = alpha
        self.l1_ratio = l1_ratio  # Used for Elastic Net
        self.weights = None
        self.bias = None
        self.loss_history = []

    def _mse(self, y, y_pred):
        """Compute Mean Squared Error."""
        return np.mean((y - y_pred) ** 2)

    @abstractmethod
    def _compute_regularization(self):
        """Compute regularization term for weights. Must be implemented by subclasses."""
        pass

    def fit(self, X, y):
        """Train the model using Gradient Descent."""
        n_samples, n_features = X.shape
        self.weights = np.zeros((n_features, 1))
        self.bias = np.zeros((1, 1))

        for _ in range(self.n_iters):
            y_pred = self._predict_raw(X)
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y)) + self._compute_regularization()
            db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

            loss = self._mse(y, y_pred)
            self.loss_history.append(loss)

    def closed_form_solution(self, X, y):
        """Compute the closed-form solution using the Normal Equation."""
        n_samples = X.shape[0]
        X_b = np.c_[np.ones((n_samples, 1)), X]  # Add bias column

        w_closed = np.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y  # Use pseudo-inverse for stability

        self.bias = w_closed[0]  # First element is bias
        self.weights = w_closed[1:]  # Remaining are weights

    def _predict_raw(self, X):
        """Compute the raw linear predictions (before activation)."""
        return np.dot(X, self.weights) + self.bias

    def predict(self, X):
        """Make predictions."""
        return self._predict_raw(X)

In [17]:
class LinearRegression(BaseRegression):
    """Standard Linear Regression (No Regularization)."""
    
    def __init__(self, learning_rate=0.01, n_iters=500):
        super().__init__(learning_rate, n_iters, alpha=0)

    def _compute_regularization(self):
        return 0  # No regularization for Linear Regression

In [18]:
class PolynomialRegression:
    """Polynomial Regression Wrapper for Any Regression Model."""
    
    def __init__(self, base_model, degree=2):
        self.base_model = base_model
        self.degree = degree
        self.poly = PolynomialFeatures(degree=degree, include_bias=False)
    
    def fit(self, X, y):
        X_poly = self.poly.fit_transform(X)
        self.base_model.fit(X_poly, y)
    
    def predict(self, X):
        X_poly = self.poly.transform(X)
        return self.base_model.predict(X_poly)

In [19]:
class RidgeRegression(BaseRegression):
    """Ridge Regression (L2 Regularization)."""
    
    def __init__(self, alpha=1.0, learning_rate=0.01, n_iters=500):
        super().__init__(learning_rate, n_iters, alpha)

    def _compute_regularization(self):
        return (self.alpha / self.weights.shape[0]) * self.weights  # L2 Regularization

In [20]:
class LassoRegression(BaseRegression):
    """Lasso Regression (L1 Regularization)."""
    
    def __init__(self, alpha=0.1, learning_rate=0.01, n_iters=500):
        super().__init__(learning_rate, n_iters, alpha)

    def _compute_regularization(self):
        return (self.alpha / self.weights.shape[0]) * np.sign(self.weights)  # L1 Regularization

In [21]:
class ElasticNetRegression(BaseRegression):
    """Elastic Net Regression (Combination of Ridge and Lasso)."""
    
    def __init__(self, alpha=1.0, l1_ratio=0.5, learning_rate=0.01, n_iters=500):
        super().__init__(learning_rate, n_iters, alpha, l1_ratio)

    def _compute_regularization(self):
        l1_term = self.l1_ratio * np.sign(self.weights)  # L1 (Lasso)
        l2_term = (1 - self.l1_ratio) * self.weights  # L2 (Ridge)
        return (self.alpha / self.weights.shape[0]) * (l1_term + l2_term)